In [7]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [19]:
data = pd.read_csv('../data/labeled_post.csv')

In [12]:
current_index = 0
show_all = False  # Flag to control what posts to show

def create_post_html(post):
    return f"""
    <div style='margin-bottom: 20px'>
        <h3>Post {current_index + 1} of {len(filtered_data())}</h3>
        <h4>Title:</h4> {post['title']}
        <h4>Body:</h4> {post['body'][:1500]}{"..." if len(str(post['body'])) > 1500 else ""}
        <h4>Subreddit:</h4> {post['subreddit']}
        <h4>Score:</h4> {post['score']}
        <h4>Comments:</h4> {post['num_comments']}
        <h4>Current labels:</h4>
        <ul>
            <li>Is Problem: {post.get('is_problem', 'Not labeled')}</li>
            <li>Problem Type: {post.get('problem_type', 'Not labeled')}</li>
            <li>Difficulty: {post.get('difficulty', 'Not labeled')}</li>
        </ul>
    </div>
    """

def filtered_data():
    if show_all:
        return data
    return data[data['has_been_labeled'] != 1]

# Create an output widget for post-content and status
post_output = widgets.HTML()
status_output = widgets.HTML()

def update_status():
    total = len(data)
    labeled = (data['has_been_labeled'] == 1).sum()
    unlabeled = total - labeled
    status_output.value = f"""
    <p>Progress: {labeled} labeled, {unlabeled} remaining
       <br>Showing: {"All Posts" if show_all else "Unlabeled Posts Only"}</p>
    """

def show_post():
    if len(filtered_data()) > 0:
        post = filtered_data().iloc[current_index]
        post_output.value = create_post_html(post)
    else:
        post_output.value = "<h3>No posts to show with current filter</h3>"
    update_status()

def on_next_clicked(b):
    print("Next button clicked")
    global current_index

    data.at[current_index, 'has_been_labeled'] = 1
    auto_save()

    if current_index < len(filtered_data()) - 1:
        current_index += 1
        show_post()

def on_prev_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
        show_post()

def on_label_change(change):
    current_post_idx = filtered_data().index[current_index]
    data.at[current_post_idx, 'is_problem'] = is_problem.value
    data.at[current_post_idx, 'problem_type'] = problem_type.value
    data.at[current_post_idx, 'difficulty'] = difficulty.value
    update_status()

def auto_save():
    print('Auto-saving progress...')
    data.to_csv('../data/labeled_post.csv', index=False)
    update_status()

def toggle_view(b):
    global current_index, show_all
    show_all = not show_all
    current_index = 0  # Reset to the first post when switching views
    toggle_button.description = 'Showing: ' + ('All' if show_all else 'Unlabeled')
    show_post()

# Create widgets
prev_button = widgets.Button(
    description='Previous',
    button_style='info',
    layout=widgets.Layout(width='100px')
)
next_button = widgets.Button(
    description='Next',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

toggle_button = widgets.Button(
    description='Showing: Unlabeled',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

is_problem = widgets.Dropdown(
    options=[0, 1],
    description='Is Problem:',
    layout=widgets.Layout(width='300px')
)

problem_type = widgets.Dropdown(
    options=[
        (0, 'Not a Problem'),
        (1, 'Software Search'),
        (2, 'Alternative Seeking'),
        (3, 'Automation Need'),
        (4, 'Feature Request'),
        (5, 'Integration/Technical')
    ],
    description='Problem Type:',
    layout=widgets.Layout(width='300px')
)

difficulty = widgets.Dropdown(
    options=[0, 1, 2, 3],
    description='Difficulty:',
    layout=widgets.Layout(width='300px')
)

# Register callbacks
prev_button.on_click(on_prev_clicked)
next_button.on_click(on_next_clicked)
toggle_button.on_click(toggle_view)
is_problem.observe(on_label_change, names='value')
problem_type.observe(on_label_change, names='value')
difficulty.observe(on_label_change, names='value')

# Create the layout
navigation = widgets.HBox([prev_button, next_button, toggle_button])
controls = widgets.VBox([is_problem, problem_type, difficulty])
layout = widgets.VBox([post_output, status_output, navigation, controls])

# Initial display
show_post()
display(layout)